In [1]:
# Allows code to live reload
%load_ext autoreload
%autoreload 2

## 1. Install
```sh
# Install prerequisites
pip install pyodc

```

## 2. Make an ECMWF account
- Go to ecmwf.int/, click login at the top right and click register to make a new account.
- Once logged in, go to api.ecmwf.int/v1/key/ to get your key. 
- Put it in `~/.ecmwfapirc` as directed.

In [2]:
# Load in the ECMWF token 
from pathlib import Path
import json
import requests
from IPython.display import JSON
from datetime import datetime as dt
from datetime import timedelta, timezone
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

with open(Path("~/.ecmwfapirc").expanduser(), "r") as f:
    api_creds = json.load(f)

print("Checking API credentials")
r = requests.get(f"https://api.ecmwf.int/v1/who-am-i?token={api_creds['key']}")
if r.status_code == 403: print("Your credentials are either wrong or need to be renewed at https://api.ecmwf.int/v1/key/")
r.raise_for_status()
JSON(r.json())

Checking API credentials


<IPython.core.display.JSON object>

In [3]:
session = requests.Session()
session.headers["Authorization"] = f"Bearer {api_creds['key']}"

In [4]:
url = "http://ionbeam-ichange.ecmwf-ichange.f.ewcloud.host/api/v1/"
url = "http://localhost:5002/api/v1/"

In [5]:
from datetime import datetime

stations = session.get(url + "stations").json()
print(f"{len(stations) = }")

len(stations) = 62


In [6]:
granules = session.get(url + "list").json()
print(f"{len(granules) = }")

len(granules) = 62


In [7]:
from collections import Counter
types = Counter(s["platform"] for s in stations)
types

Counter({'acronet': 50, 'meteotracker': 12})

In [8]:
from collections import defaultdict
by_platform = defaultdict(list)
for s in stations:
    by_platform[s["platform"]].append(s)

print("Most recently updated entry from each platform\n")
for platform, platform_stations in by_platform.items():
    print(platform)
    print(json.dumps(
        sorted(platform_stations, key = lambda s : datetime.fromisoformat(s["time_span"][1]) )[-1],
        indent = 4,
        ))

Most recently updated entry from each platform

meteotracker
{
    "name": "MeteoTracker Track",
    "description": "A MeteoTracker Track.",
    "platform": "meteotracker",
    "external_id": "6757333bdf369109189ca6d2",
    "internal_id": "6f768a105f722153",
    "location": [
        8.911294450000002,
        44.4411879
    ],
    "time_span": [
        "2024-12-09T18:13:18Z",
        "2024-12-09T18:35:44Z"
    ],
    "authors": [
        {
            "name": "meteotracker"
        },
        {
            "name": "genova_living_lab_1"
        }
    ],
    "mars_request": {
        "class": "rd",
        "expver": "xxxx",
        "stream": "lwda",
        "aggregation_type": "tracked",
        "date": "20241209",
        "platform": "meteotracker",
        "internal_id": "6f768a105f722153"
    }
}
acronet
{
    "name": "Monte Santa Croce",
    "description": "An Acronet station",
    "platform": "acronet",
    "external_id": "monte_santa_croce",
    "internal_id": "08fcafda1bb07325",

## Obtain a whole meteotracker track

For meteotracker tracks it is sufficient to simply use the "mars_request" as a key to the retrieve endpoint (along with format=csv) to download the data.

In [9]:
from io import BytesIO

example_station = by_platform["meteotracker"][-1]

args = {
    "format" : "csv"
}

data = session.get(url + "retrieve", params = example_station["mars_request"] | args)

df = pd.read_csv(BytesIO(data.content))
df

,class,expver,stream,project,platform,aggregation_type,source_name,external_id,internal_id,date,solar_radiation_index,relative_humidity_near_surface,altitude,dew_point_temperature,potential_temperature,air_temperature_near_surface,humidity_index,lat,lon,datetime
0,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,64,7.0,282.45,286.8,289.05,289.95,51.982150,5.652502,2024-12-09T18:03:36Z
1,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,65,7.0,282.25,286.4,288.65,289.45,51.982113,5.652587,2024-12-09T18:03:51Z
2,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,66,7.0,282.15,286.2,288.45,289.15,51.982124,5.652520,2024-12-09T18:04:06Z
3,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,67,7.0,281.75,285.5,287.75,288.35,51.982137,5.652511,2024-12-09T18:04:21Z
4,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,68,7.0,281.65,285.1,287.35,287.95,51.982157,5.652491,2024-12-09T18:04:37Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,99,2.0,278.85,276.6,278.85,278.25,51.982132,5.652427,2024-12-09T18:34:14Z
212,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,99,2.0,278.85,276.6,278.85,278.25,51.982124,5.652422,2024-12-09T18:34:17Z
213,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,99,2.0,278.85,276.6,278.85,278.25,51.982121,5.652422,2024-12-09T18:34:20Z
214,rd,xxxx,lwda,I-CHANGE,meteotracker,tracked,Gert-Jan Steeneveld,675730ecdf369109189af4dc,23df3b982198dec1,20241209,NaN,99,3.0,278.85,276.6,278.85,278.25,51.982118,5.652424,2024-12-09T18:34:23Z


In [10]:
import geopandas as gpd
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)
geo_df.explore(column = "altitude")

## Obtain Acronet data and other streams

For the meteotracker data, each track is stored in a single data file. For continuous streams of data like the acronet stations, it is stored in 5 minute granules.

In [11]:
[station["name"] for station in by_platform["acronet"]][:10] + [f"... + {len(by_platform["acronet"])} more"]

['Vasca Strà',
 'Sede PC AIB',
 'Asilo Tovo',
 'Porto Antico Genova',
 'Sede Comunale Moconesi',
 'Casa della Miniera',
 'Municipio Piazza V. Veneto 8',
 'Chiesa San Nicolò',
 'Scuola Stella San Giovanni',
 'Scuola Primaria Villa Sanguineti (Teglia)',
 '... + 50 more']

In [12]:
example_station = by_platform["acronet"][-1]
example_station

{'name': 'Monte Santa Croce',
 'description': 'An Acronet station',
 'platform': 'acronet',
 'external_id': 'monte_santa_croce',
 'internal_id': '08fcafda1bb07325',
 'location': [9.679447, 44.148647],
 'time_span': ['2024-12-09T12:50:00Z', '2024-12-09T12:55:00Z'],
 'authors': [{'name': 'acronet'}],
 'mars_request': {'class': 'rd',
  'expver': 'xxxx',
  'stream': 'lwda',
  'aggregation_type': 'chunked',
  'date': '20241209',
  'platform': 'acronet',
  'internal_id': '08fcafda1bb07325'}}

Giving the "mars_request" to the `list` endpoint gives the list of data granules available from the list endpoint. For the acronet data and other continuous streams, an additional key is needed "start_time". 

In [13]:
data_granules = session.get(url + "list", params = example_station["mars_request"])
sorted([g["mars_request"]["start_time"] for g in data_granules.json()])

['1250']

The `retrieve` endpoint simply concatenates the datafiles matched by a given mars request.

In [14]:
args = {
    "format" : "csv"
}

data = session.get(url + "retrieve", params = example_station["mars_request"] | args)

df = pd.read_csv(BytesIO(data.content))
df

,class,expver,stream,project,platform,aggregation_type,source_name,external_id,internal_id,date,...,signal_strength,air_temperature_near_surface,internal_temperature,battery_level,altitude,relative_humidity_near_surface,wind_direction_near_surface,wind_speed_near_surface,wind_gust,air_pressure_near_surface
0,rd,xxxx,lwda,I-CHANGE,Acronet,chunked,Acronet,monte_santa_croce,08fcafda1bb07325,20241209,...,NaN,279.35,286.65,13.52,753.579999,65.800000,254.70000,0.514444,0.823110,93486.000
1,rd,xxxx,lwda,I-CHANGE,Acronet,chunked,Acronet,monte_santa_croce,08fcafda1bb07325,20241209,...,NaN,279.35,286.65,13.52,754.112223,66.200005,204.90001,0.617333,1.543332,93480.000
2,rd,xxxx,lwda,I-CHANGE,Acronet,chunked,Acronet,monte_santa_croce,08fcafda1bb07325,20241209,...,NaN,279.25,286.45,13.54,753.579999,65.900000,333.30002,0.874555,1.388999,93486.000
3,rd,xxxx,lwda,I-CHANGE,Acronet,chunked,Acronet,monte_santa_croce,08fcafda1bb07325,20241209,...,22.0,279.25,286.35,13.52,753.757670,66.400000,249.00000,0.823110,1.954887,93483.997
4,rd,xxxx,lwda,I-CHANGE,Acronet,chunked,Acronet,monte_santa_croce,08fcafda1bb07325,20241209,...,NaN,279.15,286.25,13.53,753.934812,66.500000,233.10000,0.565888,1.286110,93482.000
5,rd,xxxx,lwda,I-CHANGE,Acronet,chunked,Acronet,monte_santa_croce,08fcafda1bb07325,20241209,...,NaN,279.25,286.25,13.52,753.757670,67.400000,283.00000,0.514444,1.131777,93483.997
